# Advanced Class Topics

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.getcwd()
os.chdir('/Users/fizz/Document/Notes/Python/codes')

In [ ]:
# Extending Types by Embedding
class Set:
    def __init__(self, value=[]):
        self.data = []
        self.concat(value)
    def intersect(self, other):
        res = []
        for x in self.data:
            if x in other:
                res.append(x)
        return Set(res)
    def union(self, other):
        res = self.data[:]
        for x in other:
            if not x in res:
                res.append(x)
        return res
    def concat(self, value):
        for x in value:
            if not x in self.data:
                self.data.append(x)
    def __len__(self): return len(self.data)
    def __getitem__(self, key): return self.data[key]
    def __and__(self, other): return self.intersect(other)
    def __or__(self, other): return self.union(other)
    def __repr__(self): return 'Set:' + repr(self.data)
    def __iter__(self): return iter(self.data)

In [18]:
# Extending Types by Subclassing
class Set(list):
    def __init__(self, value=[]):
        self.concat(value)
    def intersect(self, other):
        res = []
        for x in self:
            if x in other:
                res.append(x)
        return Set(res)
    def union(self, other):
        res = Set(self)
        res.concat(other)
        return res
    def concat(self, value):
        for x in value:
            if not x in self:
                self.append(x)
    def __and__(self, other): return self.intersect(other)
    def __or__(self, other): return self.union(other)
    def __repr__(self): return 'Set:' + list.__repr__(self)

x = Set([1,3,5,7])
y = Set([2,1,4,5,6])
print(x, y, len(x)) 
print(x.intersect(y), y.union(x)) 
print(x & y, x | y)
x.reverse(); print(x)

Set:[1, 3, 5, 7] Set:[2, 1, 4, 5, 6] 4
Set:[1, 5] Set:[2, 1, 4, 5, 6, 3, 7]
Set:[1, 5] Set:[1, 3, 5, 7, 2, 4, 6]
Set:[7, 5, 3, 1]


## The 'New Style' Class Model

In Python 3.X, all classes are automatically what were formerly called "new style," whether they explicitly inherit from __objec__ or not. Coding the __objec__ superclass is optional and implied.

* Attribute Fetch for Built-in Skips Instances  
 In new-style classes (and hence all classes in 3.X), the generic instance attribute interception methods **\_\_getattr\_\_** and **\_\_getattribute\_\_** are no longer called by built-in operations for **\_\_X\_\_** operation overloading method names - the search for such names begins at classes, not instances. Attributes accessed by explicit name, however, are routed through these method, even if they are **\_\_X\_\_** names. Hence, this is primarily a change to the behavior of built-in operations.

In [27]:
class C:
    def __getattr__(self, name): print(name)
X = C()
X.normal
X.__add__
X + 1

normal
__add__


TypeError: unsupported operand type(s) for +: 'C' and 'int'

In [42]:
class C:
    def __getattr__(self, name):
        print(name)
X = C()
X.normal
X.__add__
X + 1

normal
__add__


AttributeError: type object 'C' has no attribute '__add__'

In [53]:
class C:
    data = 'spam'
    def __getattr__(self, name):
        print('getattr: ' + name)
        return getattr(self.data, name)
    def __getitem__(self, i):
        print('getitem: ' + str(i))
        return self.data[i]
    def __add__(self, other):
        print('add: ' + other)
        return getattr(self.data, '__add__')(other)
X = C()
X.upper()
X[1]
type(X).__getitem__(X, 1)

getattr: upper


'SPAM'

getitem: 1


'p'

getitem: 1


'p'

Type Model Changes  
* Classes are types  
 The __type__ object generates classes as its instances, and classes generate instances of themselves. Both are considered types, because they generate instances. In fact, there is no real difference between built-in types like lists and strings and user-defined types coded as classes.
 
* Types are classes  
  ......

In [57]:
class C: pass
I = C()
type(I), I.__class__
type(C), C.__class__
type([1, 2, 3]), [1, 2, 3].__class__
type(list), list.__class__

isinstance(X, object)
isinstance(C, object)

(__main__.C, __main__.C)

(type, type)

(list, list)

(type, type)

True

True

In [59]:
type(type), type(object)
isinstance(type, object), isinstance(object, type)
type is object

(type, type)

(True, True)

False

Diamond Inheritance Change  
* For classic classes (the default in 2.X): __DFLR__  
 The inheritance search path is strictly depth first, and then left to right - Python climbs all the way to the top, hugging the left side of the tree, before it backs up and begins to look further to the right.  
* For new-style classes (optional in 2.X and automatic in 3.X): __MRO__  
 The inheritance search path is more breadth-first in diamond cases - Python first looks in any superclasses to the right of the one just searched before ascending to the common superclass at the top. In other words, this search proceeds across by levels before moving up. This search order is called the new-style _MRO_ for "method resolution order". Despite the name, this is used for all attributes in Python, not just methods.

In [63]:
class A: attr = 1
class B(A): pass
class C(A): attr = 2
class D(B, C): pass
X = D()       # Searches X, D, B, C
X.attr

2

In [64]:
class A: attr = 1
class B(A): pass
class C: attr = 2
class D(B, C): pass
X = D()
X.attr

1

In [76]:
# Tracing the MRO
class A: attr = 1
class B(A): pass
class C(A): attr = 2
class D(B, C): pass
D.__mro__

# Nondiamonds
class A: attr = 1
class B(A): pass
class C: attr = 2
class D(B, C): pass
D.__mro__

class A: attr = 1
class B: pass
class C(A): attr = 2
class D(B, C): pass
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

(__main__.D, __main__.B, __main__.A, __main__.C, object)

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [79]:
D.mro() == list(D.__mro__)
[cls.__name__ for cls in D.__mro__]

True

['D', 'B', 'C', 'A', 'object']

In [ ]:
"""
File mapattrs.py (3.X + 2.X)
Main tool: mapattrs() maps all attributes on or inherited by an instance to the instance or class from which they are inherited.
Assumes dir() gives all attributes of an instance. To simulate inheritance, uses either the class's MRO tuple, which gives the search order for new-style classes (and all in 3.X), or a recursive traversal to infer the DFLR order of classic classes in 2.X.
Also here: inheritance() gives version-neutral class ordering; assorted dictionary tools using 3.X/2.7 comprehensions.
"""
import pprint
def trace(X, label='', end='\n'):
    print(label + pprint.pformat(X) + end)    # Print nicely
def filterdictvals(D, V):
    """
    dict D with entries for value V removed. 
    filterdictvals(dict(a=1, b=2, c=1), 1) => {'b': 2} 
    """
    return {K:V2 for K, V2 in D.items() if V2 != V}
def invertdict(D):
    """
    dict D with values changed to keys (grouped by values).
    Values must all be hashable to work as dict/set keys.
    inverdict(dict(a=1, b=2, c=1)) => {1: ['a', 'c'], 2: ['b']}
    """
    def keysof(V):
        return sorted(K for K in D.keys if D[K] == V)
    return {V:keysof(V) for V in set(D.values())}
def dflr(cls):
    """
    Classic depth-first left-to-right order of class tree at cls.
    Cycles not possible: Python disallows on __bases__ changes.
    """
    here = [cls]
    for sup in cls.__bases__:
        here += dflr(sup)
    return here
def inheritance(instance):
    """
    Inheritance order sequence: new-style (MRO) or classic (DFLR)
    """
    if hasattr(instance.__class__, '__mro__'):
        return (instance,) + instance.__class__.__mro__
    else:
        return [instance] + dflr(instance.__class__)
def mapattrs(instance, withobject=False, bysource=False):
    """
    dict with keys giving all inherited attributes of instance, with values giving the object that
    each is inherited from.
    withobject: False=remove object built-in class attributes.
    bysource: True=group result by objects instead of attributes.
    Supports classes with slots that preclude __dict__ in instances.
    """
    attr2obj = {}
    inherits = inheritance(instance)
    for attr in dir(instance):
        for obj in inherits:
            if hasattr(obj, '__dict__') and attr in obj.__dict__:      # See slots
                attr2obj[attr] = obj
                break
    if not withobject:
        attr2obj = filterdictvals(attr2obj, object)
    return attr2obj if not bysource else invertdict(attr2obj)

if __name__ == '__main__':
    print('Classic classes in 2.X, new-style in 3.X')
    class A: attr1 = 1
    class B(A): attr2 = 2
    class C(A): attr1 = 3
    class D(B, C): pass
    I = D()
    print('Py=>%s' % I.attr1)
    ...

New-Style Class Extensions  
* Slot basics  
 To use slots, assign a sequence of stirng names to the special **\_\_slots\_\_** variable and attribute at the top level of a __class__ statement: only those names in the **\_\_slots\_\_** list can be assigned as instance attributes.

In [84]:
class limiter:
    __slots__ = ['age', 'name', 'job']
x = limiter()
x.age = 40
x.age
x.ape = 1000    # not in __slots__

40

AttributeError: 'limiter' object has no attribute 'ape'

In [86]:
class C:
    __slots__ = ['a', 'b']    # __slots__ means no __dict__ by default
X = C()
X.a = 1
X.a
getattr(X, 'a')                # But getattr() and setattr() still work
setattr(X, 'b', 2)
'a' in dir(X)                   # And dir() finds slot attributes too
'b' in dir(X)
X.__dict__

1

1

True

True

AttributeError: 'C' object has no attribute '__dict__'

In [87]:
# Also keep in mind that without an attribute namespace dictionary, it's not possible to assign
# new names to instances that are not in the slots list:
class D:
    __slots__ = ['a', 'b']
    def __init__(self):
        self.d = 4
X = D()             # Cannot add new names if no __dict__

AttributeError: 'D' object has no attribute 'd'

In [88]:
# We can still accommodate extra attributes, though, by including __dict__ explicitly in __slots__.
# in order to create an attribute namespace dictionary too:
class D:
    __slots__ = ['a', 'b', '__dict__']       # Name __dict__ to include one too
    c = 3                                        # Class attrs work normally
    def __init__(self):
        self.d = 4
X = D()
X.d

4

P1013 To Be Continued

In [7]:
game()

jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!

7

In [18]:
game()

yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!

7

In [19]:
game()

jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!

8

In [24]:
game()

yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!

4

In [25]:
game()

jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!

6

In [80]:
game()

yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!

4

In [81]:
game()

jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!

6

In [82]:
game()

yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!

7

In [83]:
game()

jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!
jiayu win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
jiayu win!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
jiayu has longer legs than yiyun's!
jiayu win!
jiayu win!
yiyun win!
yiyun win!
jiayu win!
jiayu has longer legs than yiyun's!
yiyun win!
jiayu win!
jiayu win!
jiayu win!
yiyun win!
jiayu has longer legs than yiyun's!
yiyun win!
yiyun win!
yiyun win!
yiyun win!
jiayu win!
yiyun has longer legs than jiayu's!
jiayu win!
yiyun win!
jiayu win!
yiyun win!
yiyun win!
yiyun has longer legs than jiayu's!

8